In [132]:
import numpy as np
import pickle
import scipy.stats as st
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

In [135]:
XX = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\State_Data_Nominal.npy',allow_pickle=True)
UU = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\Input_Dataset_new.npy',allow_pickle=True)

In [134]:
X = []
X_plot = []

X_Prev = []
X_plot_Prev = []

U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i]))
    X_plot.append(np.array(XX[i]))

X = np.concatenate(X, axis=0)
X = np.reshape(X, (-1, 2))

U = []
for i in range(len(UU)):
    U.append(np.array(UU[i]))
    U_plot.append(np.array(UU[i]))

U = np.concatenate(U, axis=0)
U = np.reshape(U, (-1, 2))

In [131]:
X1 = XX[0,:,:,:]
X2 = XX[:2,:,:,:].reshape((-1,2))
X3 = XX[:,:,:,:].reshape((-1,2))

U1 = UU[0,:,:,:]
U2 = UU[:2,:,:,:].reshape((-1,2))
U3 = UU[:,:,:,:].reshape((-1,2))

In [169]:
# Dataset 1
X1 = np.hstack((XX[0,:-1])).reshape(-1,2)
U1 = np.hstack((UU[0,:])).reshape(-1,2)
Train1 = np.hstack((X1,U1))
Y1 = np.hstack((XX[0,1:])).reshape(-1,2)

# Dataset 2
X2 = np.hstack((XX[0,:-1],XX[1,:-1])).reshape(-1,2)
U2 = np.hstack((UU[0,:],UU[1,:])).reshape(-1,2)
Train2 = np.hstack((X2,U2))
Y2 = np.hstack((XX[0,1:],XX[1,1:])).reshape(-1,2)

# Dataset 3
X3 = np.hstack((XX[0,:-1],XX[1,:-1],XX[2,:-1])).reshape(-1,2)
U3 = np.hstack((UU[0,:],UU[1,:],UU[2,:])).reshape(-1,2)
Train3 = np.hstack((X3,U3))
Y3 = np.hstack((XX[0,1:],XX[1,1:],XX[2,1:])).reshape(-1,2)

In [ ]:
# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.0,.0,.0,.0]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(Train2, Y2)

In [ ]:
X_train = np.hstack((X2[:-1,:],U2[:,:])).reshape((-1,4))
Y_train = X2[1:,:].reshape((-1,2))

# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.5,0.5,0.5,0.5]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(X_train, Y_train)


In [ ]:
gp.kernel_.get_params()

In [113]:
with open("GP_nominal_2.dump","wb") as f:
    pickle.dump(gp, f)